# Anomaly detection with K-means Clustering 

Lab Exercises:
1. Implement a PySpark script to handle any missing values and scale numerical features.
2. Develop a PySpark script that uses the K-means algorithm to cluster data points.
3. Develop a PySpark script that labels data points as anomalies based on their cluster assignments.
4. Implement code to evaluate the effectiveness of the K-means clustering model in detecting anomalies.

**Importing necessary modules and setting environment variables for PySpark to use the correct Python executable. Importing `SparkContext` from `pyspark` and `SparkSession` from `pyspark.sql`.**


In [ ]:
import pyspark
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql import SparkSession

**Initializing a SparkSession named 'chapter_5' with a driver memory of 16 GB using the `SparkSession.builder.config()` method.**


In [ ]:
spark = SparkSession.builder.config("spark.driver.memory", "16g").appName('chapter_5').getOrCreate()

## A First Take on Clustering

**Reading a CSV file located at "data/kddcup.data_10_percent_corrected" into a Spark DataFrame `data_without_header`, treating the first row as data rather than a header. Then assigning column names to the DataFrame based on a predefined list `column_names`. Finally, creating a new DataFrame `data` with the specified column names.**

In [ ]:
data_without_header = spark.read.option("inferSchema", True).option("header", False).csv("data/kddcup.data_10_percent_corrected")

column_names = [ "duration", "protocol_type", "service", "flag",
  "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
  "hot", "num_failed_logins", "logged_in", "num_compromised",
  "root_shell", "su_attempted", "num_root", "num_file_creations",
  "num_shells", "num_access_files", "num_outbound_cmds",
  "is_host_login", "is_guest_login", "count", "srv_count",
  "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
  "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
  "dst_host_count", "dst_host_srv_count",
  "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
  "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
  "dst_host_serror_rate", "dst_host_srv_serror_rate",
  "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
  "label"]

data = data_without_header.toDF(*column_names)

**Selecting the 'label' column from the DataFrame `data`, grouping it by unique values in the 'label' column, counting the occurrences of each label, ordering the result by the count in descending order, and displaying the top 25 labels along with their counts.**


In [ ]:
from pyspark.sql.functions import col

data.select("label").groupBy("label").count().orderBy(col("count").desc()).show(25)

**Importing necessary modules and classes from PySpark for performing K-means clustering. First dropping non-numeric columns from the DataFrame `data` and caching the resulting DataFrame `numeric_only`. Then, setting up a `VectorAssembler` to assemble feature vectors from numeric columns, followed by initializing a K-means model. A pipeline is constructed to execute these transformations and modeling steps sequentially. After fitting the pipeline to the data, the cluster centers from the trained K-means model are printed using the `clusterCenters()` method.**

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml import Pipeline

numeric_only = data.drop("protocol_type", "service", "flag").cache()

assembler = VectorAssembler().setInputCols(numeric_only.columns[:-1]).setOutputCol("featureVector")

kmeans = KMeans().setPredictionCol("cluster").setFeaturesCol("featureVector")

pipeline = Pipeline().setStages([assembler, kmeans])
pipeline_model = pipeline.fit(numeric_only)
kmeans_model = pipeline_model.stages[1]

from pprint import pprint
pprint(kmeans_model.clusterCenters())

**Applying the trained pipeline model `pipeline_model` to the `numeric_only` DataFrame, generating predictions and adding a 'cluster' column to the DataFrame `with_cluster`. Then selecting the 'cluster' and 'label' columns, groups by both columns, counting the occurrences of each combination, ordering the result first by cluster and then by count in descending order, and displaying the top 25 combinations of clusters and labels along with their counts.**


In [ ]:
with_cluster = pipeline_model.transform(numeric_only)

with_cluster.select("cluster", "label").groupBy("cluster", "label").count().orderBy(col("cluster"), col("count").desc()).show(25)

## Choosing k

**Defining a function `clustering_score(input_data, k)` to calculate the training cost of K-means clustering for a given value of `k` clusters. Within the function, dropping non-numeric columns, setting up a pipeline with a `VectorAssembler` and a K-means model, fitting the pipeline to the data, and retrieving the training cost from the K-means model summary.**

**Then, iterating over a range of `k` values from 20 to 80 (inclusive), incrementing by 20, and printing the training cost for each `k`.**


In [ ]:
from pyspark.sql import DataFrame
from random import randint

def clustering_score(input_data, k):
  input_numeric_only = input_data.drop("protocol_type", "service", "flag")
  assembler = VectorAssembler().setInputCols(input_numeric_only.columns[:-1]).setOutputCol("featureVector")
  kmeans = KMeans().setSeed(randint(100,100000)).setK(k).setPredictionCol("cluster").setFeaturesCol("featureVector")
  pipeline = Pipeline().setStages([assembler, kmeans])
  pipeline_model = pipeline.fit(input_numeric_only)
  kmeans_model = pipeline_model.stages[-1]
  training_cost = kmeans_model.summary.trainingCost
  return training_cost

for k in list(range(20,100, 20)):
  print(clustering_score(numeric_only, k))

**Defining a modified version of the `clustering_score` function called `clustering_score_1`. Calculating the training cost of K-means clustering for a given value of `k` clusters. Within the function, dropping non-numeric columns, setting up a pipeline with a `VectorAssembler` and a K-means model with additional parameters (`setMaxIter`, `setTol`), and fitting the pipeline to the data.**

**Then, iterating over a range of `k` values from 20 to 100 (inclusive), incrementing by 20, and printing the `k` value along with the corresponding training cost calculated by the `clustering_score_1` function.**


In [ ]:
def clustering_score_1(input_data, k):
  input_numeric_only = input_data.drop("protocol_type", "service", "flag")
  assembler = VectorAssembler().setInputCols(input_numeric_only.columns[:-1]).setOutputCol("featureVector")
  kmeans = KMeans().setSeed(randint(100,100000)).setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster").setFeaturesCol("featureVector")
  pipeline = Pipeline().setStages([assembler, kmeans])
  pipeline_model = pipeline.fit(input_numeric_only)
  kmeans_model = pipeline_model.stages[-1]
  training_cost = kmeans_model.summary.trainingCost
  return training_cost

for k in list(range(20,101, 20)):
  print(k, clustering_score_1(numeric_only, k))

## Feature Normalization

**Defining another modified version of the `clustering_score` function called `clustering_score_2`. Calculating the training cost of K-means clustering for a given value of `k` clusters. Within the function, dropping non-numeric columns, setting up a pipeline with a `VectorAssembler`, a `StandardScaler`, and a K-means model with additional parameters (`setMaxIter`, `setTol`), and fitting the pipeline to the data.**

**Then, iterating over a range of `k` values from 60 to 270 (inclusive), incrementing by 30, and printing the `k` value along with the corresponding training cost calculated by the `clustering_score_2` function.**


In [ ]:
from pyspark.ml.feature import StandardScaler

def clustering_score_2(input_data, k):
  input_numeric_only = input_data.drop("protocol_type", "service", "flag")
  assembler = VectorAssembler().setInputCols(input_numeric_only.columns[:-1]).setOutputCol("featureVector")
  scaler = StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").setWithStd(True).setWithMean(False)
  kmeans = KMeans().setSeed(randint(100,100000)).setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster").setFeaturesCol("scaledFeatureVector")
  pipeline = Pipeline().setStages([assembler, scaler, kmeans])
  pipeline_model = pipeline.fit(input_numeric_only)
  kmeans_model = pipeline_model.stages[-1]
  training_cost = kmeans_model.summary.trainingCost
  return training_cost

for k in list(range(60, 271, 30)):
  print(k, clustering_score_2(numeric_only, k))

## Categorical Variables

**Defining a function `one_hot_pipeline(input_col)` to create a pipeline for one-hot encoding a categorical column. Within the function, setting up a `StringIndexer` to index the categorical column and a `OneHotEncoder` to encode the indexed column. These transformations are encapsulated in a pipeline.**

**The function returns the pipeline and the name of the output column after one-hot encoding.**


In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

def one_hot_pipeline(input_col):
  indexer = StringIndexer().setInputCol(input_col).setOutputCol(input_col + "_indexed")
  encoder = OneHotEncoder().setInputCol(input_col + "_indexed").setOutputCol(input_col + "_vec")
  pipeline = Pipeline().setStages([indexer, encoder])
  return pipeline, input_col + "_vec"

**Defining the function `clustering_score_3(input_data, k)` to calculate the training cost of K-means clustering for a given value of `k` clusters, considering one-hot encoded categorical features. Within the function, setting pipelines for one-hot encoding of categorical columns ("protocol_type", "service", "flag"), assembling all relevant columns, applying standard scaling, and training a K-means model.**

**Then, iterating over a range of `k` values from 60 to 270 (inclusive), incrementing by 30, and printing the `k` value along with the corresponding training cost calculated by the `clustering_score_3` function.**


In [ ]:
def clustering_score_3(input_data, k):
  proto_type_pipeline, proto_type_vec_col = one_hot_pipeline("protocol_type")
  service_pipeline, service_vec_col = one_hot_pipeline("service")
  flag_pipeline, flag_vec_col = one_hot_pipeline("flag")

  assemble_cols = set(input_data.columns) - {"label", "protocol_type", "service", "flag"} | {proto_type_vec_col, service_vec_col, flag_vec_col}

  assembler = VectorAssembler().setInputCols(list(assemble_cols)).setOutputCol("featureVector")
  scaler = StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").setWithStd(True).setWithMean(False)
  kmeans = KMeans().setSeed(randint(100,100000)).setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster").setFeaturesCol("scaledFeatureVector")
  pipeline = Pipeline().setStages([proto_type_pipeline, service_pipeline, flag_pipeline, assembler, scaler, kmeans])
  pipeline_model = pipeline.fit(input_data)

  kmeans_model = pipeline_model.stages[-1]
  training_cost = kmeans_model.summary.trainingCost
  return training_cost

for k in list(range(60, 271, 30)):
  print(k, clustering_score_3(data, k))

## Using Labels with Entropy

**Defining a function `entropy(counts)` to calculate the entropy of a list of counts. Within the function, calculating the probability of each count, computing the entropy using the formula `-p * log(p)`, and suming the results. The function returns the entropy value.**


In [ ]:
from math import log

def entropy(counts):
  values = [c for c in counts if (c > 0)]
  n = sum(values)
  p = [v/n for v in values]
  return sum([-1*(p_v) * log(p_v) for p_v in p])

**Computing the weighted average of cluster entropy for the provided data. First transforming the data using the trained pipeline model to obtain cluster labels. Then, calculating the count of each label within each cluster and ordering the result by cluster. After that, computing the probability of each label count within each cluster and adding a new column for the probability.**

**Next, calculating the entropy for each cluster by summing the product of the probability and the log probability for each label count within the cluster. Then, computing the weighted cluster entropy by multiplying the entropy with the cluster size.**

**Finally, aggregating the weighted cluster entropy across all clusters and dividing it by the total number of data points to get the weighted average cluster entropy.**


In [ ]:
from pyspark.sql import functions as fun
from pyspark.sql import Window

cluster_label = pipeline_model.transform(data).select("cluster", "label")

df = cluster_label.groupBy("cluster", "label").count().orderBy("cluster")

w = Window.partitionBy("cluster")

p_col = df['count'] / fun.sum(df['count']).over(w)
with_p_col = df.withColumn("p_col", p_col)

result = with_p_col.groupBy("cluster").agg((-fun.sum(col("p_col") * fun.log2(col("p_col")))).alias("entropy"),fun.sum(col("count")).alias("cluster_size"))

result = result.withColumn('weightedClusterEntropy',fun.col('entropy') * fun.col('cluster_size'))

weighted_cluster_entropy_avg = result.agg(fun.sum(col('weightedClusterEntropy'))).collect()
weighted_cluster_entropy_avg[0][0]/data.count()

**Defining a function `fit_pipeline_4(data, k)` to fit a pipeline for K-means clustering with one-hot encoding of categorical features and standard scaling. First setting up pipelines for one-hot encoding of categorical columns ("protocol_type", "service", "flag"). Then, assembling relevant columns, applies standard scaling, and training a K-means model using the specified number of clusters (`k`).**

**Another function `clustering_score_4(input_data, k)` is defined to calculate the weighted average cluster entropy for a given dataset and number of clusters. Within the function, fitting the pipeline to the input data, transforming the data to obtain cluster labels, calculating the cluster entropy, and computing the weighted average cluster entropy.**

**Both functions are designed to enhance modularity and readability, making it easier to fit the pipeline and calculate the clustering score.**


In [ ]:
def fit_pipeline_4(data, k):
  (proto_type_pipeline, proto_type_vec_col) = one_hot_pipeline("protocol_type")
  (service_pipeline, service_vec_col) = one_hot_pipeline("service")
  (flag_pipeline, flag_vec_col) = one_hot_pipeline("flag")
  assemble_cols = set(data.columns) - {"label", "protocol_type", "service", "flag"} | {proto_type_vec_col, service_vec_col, flag_vec_col}
  assembler = VectorAssembler(inputCols=list(assemble_cols), outputCol="featureVector")

  scaler = StandardScaler(inputCol="featureVector", outputCol="scaledFeatureVector", withStd=True, withMean=False)

  kmeans = KMeans(seed=randint(100, 100000), k=k, predictionCol="cluster", featuresCol="scaledFeatureVector", maxIter=40, tol=1.0e-5)

  pipeline = Pipeline(stages=[proto_type_pipeline, service_pipeline, flag_pipeline, assembler, scaler, kmeans])
  return pipeline.fit(data)

def clustering_score_4(input_data, k):
  pipeline_model = fit_pipeline_4(input_data, k)
  cluster_label = pipeline_model.transform(input_data).select("cluster", "label")

  df = cluster_label.groupBy("cluster", "label").count().orderBy("cluster")

  w = Window.partitionBy("cluster")

  p_col = df['count'] / fun.sum(df['count']).over(w)
  with_p_col = df.withColumn("p_col", p_col)

  result = with_p_col.groupBy("cluster").agg(-fun.sum(col("p_col") * fun.log2(col("p_col"))).alias("entropy"),fun.sum(col("count")).alias("cluster_size"))

  result = result.withColumn('weightedClusterEntropy', col('entropy') * col('cluster_size'))

  weighted_cluster_entropy_avg = result.agg(fun.sum(col('weightedClusterEntropy'))).collect()
  return weighted_cluster_entropy_avg[0][0] / input_data.count()

## Clustering in Action

**Fitting the pipeline model for K-means clustering with 180 clusters to the provided data and then applying the trained model to the data to obtain cluster labels. Selecting the 'cluster' and 'label' columns, grouping them by cluster and label, calculating the count of each combination, and ordering the result by cluster and label. Finally, displaying the resulting DataFrame showing the counts of each label within each cluster.**


In [ ]:
pipeline_model = fit_pipeline_4(data, 180)
count_by_cluster_label = pipeline_model.transform(data).select("cluster", "label").groupBy("cluster", "label").count().orderBy("cluster", "label")
count_by_cluster_label.show()